This is script will figure out travel times from different runs

In [7]:
import pandas
def readReferenceRun(file):
    return pandas.read_csv(file)

def readComparisonRuns(files):
    result = {}
    for file in files.items():
        
        run = pandas.read_csv(file[1])
        result[file[0]] = run
        
    return result

In [22]:
dataFolder = "C:\\Users\\Janek\\Desktop\\nemo_analysis\\"
baseCase = readReferenceRun(dataFolder + "base-case.csv")
policyCases = readComparisonRuns({"Radschnellwege": dataFolder + "bike-highways.csv"})
    
# get the overall travel time
baseCase['travelTime'] = baseCase['endTime'] - baseCase['startTime']
baseCase['travelTime'].sum() / baseCase.shape[0]
    




2019.962962417205